In [20]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import sqlalchemy
import os

In [21]:
conn = psycopg2.connect(
    user="dcat", 
    password="dcat", 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [ ]:
# vault login -address="https -method=oidc
# vault read --address="https://vault.adeo.no" postgresql/preprod-fss/creds/datacatalog-term-admin

conn = psycopg2.connect(
    user="v-oidc-Pau-datacata-8M61CzhFYnWwyGe4RT5T-1573823273", 
    password="A1a-uUUBJHlTN9oyrAQB", 
    host="b27dbvl008.preprod.local", 
    port="5432", 
    database="datacatalog-term")

In [22]:
cur = conn.cursor()

In [23]:
print ( conn.get_dsn_parameters(),"\n")

{'user': 'dcat', 'dbname': 'dcat', 'host': '127.0.0.1', 'port': '54320', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [24]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 12.0 (Debian 12.0-2.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



In [28]:
cur.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")

In [18]:
conn.commit()

In [29]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("drop table if exists tbl_edge")

# Node table

In [30]:
cur.execute("drop table if exists tbl_node")
cur.execute("""
create table tbl_node 
(  
  id UUID PRIMARY KEY DEFAULT gen_random_uuid(),  
  prop JSONB, 
  created TIMESTAMP default now() 
);  
""")

In [31]:
sql = "select * from tbl_node"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,prop,created


# Edge description/attributes

In [32]:
cur.execute("drop table if exists tbl_edge_desc")
cur.execute("""
create table tbl_edge_desc  -- Relationship description  
(  
  id UUID PRIMARY KEY DEFAULT gen_random_uuid(), 
  prop TEXT 
);
""")

In [33]:
sql = "select * from tbl_edge_desc"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,prop


# Edges 

In [34]:
cur.execute("drop table if exists tbl_edge")
cur.execute("""
create table tbl_edge     -- What is relationship between ID1 to ID2      
(  
  n1 uuid references tbl_node(id),    
  n2 uuid references tbl_node(id),    
  prop uuid[],  -- Multiple relationships may exist. Therefore, we use array storage. This is an edge. 
  created timestamp default now(),  
  check (n1<>n2),  
  unique (n1,n2)  
  -- FOREIGN KEY (EACH ELEMENT OF prop) REFERENCES tbl_er_desc(id)  -- Array foreign key, which is supported for PostgreSQL 11. The performance is good  
);  
""")

In [35]:
sql = "select * from tbl_edge"
df = sqlio.read_sql_query(sql, conn)
df.head()

,n1,n2,prop,created


In [36]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("create index idx_tbl_edge_n1 on tbl_edge(n1)")

In [37]:
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("create index idx_tbl_edge_n2 on tbl_edge(n2)")

In [38]:
conn.commit()